In [ ]:
#parse parma using arg parse
#connect to dbwith statement and create db_manager
#call db manager .get_table_defination_for_prompt() to get tables in prompt ready form
#create two blank calls to llm.add_cap_self () that update our current prompt based  in from cli
#call llm.prompt  to get prompt_response variable
#parse sql response   from promt response using SQL_QUER_DELIMETER '_______'
#call db_manager.run_sql()with parsed sql
 

In [20]:
import pandas as pd
from openai import OpenAI
import ast

In [2]:
import os
import dotenv
import psycopg2
from psycopg2 import sql
dotenv.load_dotenv("F:\\AI_agent_DA\\env.txt")

connection_dtl = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'dbname': os.getenv('DB_NAME')
}

def db_connect(connection_dtl):
    
    connection = psycopg2.connect(
    host=connection_dtl['host'],
    port=connection_dtl['port'],
    user=connection_dtl['user'],
    password=connection_dtl['password'],
    dbname=connection_dtl['dbname']
    )
    return connection

def execute_sql_query(connection_dtl, query):
    connection = db_connect(connection_dtl=connection_dtl)
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cols = [cursor.description[col][0] for col in range(len(cursor.description))]
        connection.close()
        return pd.DataFrame(result, columns=cols)
    except Exception as e:
        print("SQL Error")
        return pd.DataFrame()
        
llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-4"

os.environ["OPENAI_API_KEY"] = "sk-proj-jG7LIu3QAIjdDWxUUFoQT3BlbkFJ1Uy6oLUaWMsAufC4Oz4m"
client = OpenAI()

def get_completion(prompt, model=llm_model):
    
    response = client.chat.completions.create(
    model=llm_model,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )

    return response.choices[0].message.content
def retry_operation(max_attempts):
    """
    Decorator that retries a function if an exception occurs, up to a maximum number of attempts.

    Args:
    - max_attempts (int): Maximum number of attempts.
    """
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"Attempt {attempts}/{max_attempts} failed: {e}")
                    if attempts >= max_attempts:
                        print("Maximum retry attempts reached, failing operation.")
                        raise
        return wrapper
    return decorator

In [27]:
#code fetch data
import os
import psycopg2
import pandas as pd
import openai

# Database connection details
connection_dtl = {
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'dbname': os.getenv('DB_NAME')
}

# OpenAI API key setup
openai.api_key = os.getenv("OPENAI_API_KEY")


def db_connect(connection_dtl):
    connection = psycopg2.connect(
        host=connection_dtl['host'],
        port=connection_dtl['port'],
        user=connection_dtl['user'],
        password=connection_dtl['password'],
        dbname=connection_dtl['dbname']
    )
    return connection

def execute_sql_query(connection_dtl, query):
    connection = db_connect(connection_dtl=connection_dtl)
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        cols = [cursor.description[col][0] for col in range(len(cursor.description))]
        connection.close()
        return pd.DataFrame(result, columns=cols)
    except Exception as e:
        print("SQL Error")
        return pd.DataFrame()

def fetch_weekly_orders(connection_dtl, table_name):
    query = f"""
    SELECT * 
    FROM {table_name}
    """
    return execute_sql_query(connection_dtl, query)

llm_model = "gpt-4o-mini"
client = OpenAI()
def call_ai(prompt,llm_model):

    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "user", "content": prompt}
        ]
        )
    return response.choices[0].message.content

def analyse_with_ai(df):
    prompt = f"""
        *** Respond strictly with same JSON structure,***
        you are data analyst and your task is find average of total_value (it is a float8 column which contain numbers) : ```'{df}'```;"""
    response =call_ai(prompt,llm_model)
    response = ast.literal_eval(response)
    
    return response

# Main function to execute
# def perform_weekly_analysis(table_name):
#     # Fetch weekly data
#     weekly_data = fetch_weekly_orders(connection_dtl, table_name)
    
#     if weekly_data.empty:
#         return "No data available for the selected week."
    
#     # Get analysis from OpenAI
#     # ai_analysis = analyze_weekly_orders_with_ai(weekly_data)
#     return ai_analysis

# Example of usage
# result = perform_weekly_analysis("e_commerce_mini.orders")  # You can pass any table name here
# print(result)


In [28]:
table_name = 'orders'
D = fetch_weekly_orders(connection_dtl,table_name)
D

,order_id,user_id,date,total_price,payment_method,shipping_address,zip_code
0,ORDER_1,CUSTOMER_4235,2021-02-08,593.0,Credit Card,"053, Brahma Samudram, Ananthapur, ANDHRA PRADESH",112448
1,ORDER_10,CUSTOMER_6,2023-07-03,4545.0,Credit Card,"628, Gushkara, Bardhaman, WEST BENGAL",112519
2,ORDER_100,CUSTOMER_1171,2022-09-30,2397.0,Cash on Delivery,"892, Paramthivelur, Namakkal, TAMIL NADU",110835
3,ORDER_1000,CUSTOMER_2777,2023-09-16,71384.0,Credit Card,"399, Gohad, Bhind, MADHYA PRADESH",112812
4,ORDER_10000,CUSTOMER_1774,2021-05-07,2791.0,Credit Card,"1237, Sadar, Gorakhpur, UTTAR PRADESH",110048
...,...,...,...,...,...,...,...
44982,ORDER_9995,CUSTOMER_3948,2022-10-26,55283.0,Debit Card,"118, Ganapavaram Mandal, West Godavari, ANDHRA...",111481
44983,ORDER_9996,CUSTOMER_1209,2021-07-24,3349.0,Wallet,"0542, Ambabhona, Bargarh, ODISHA",111937
44984,ORDER_9997,CUSTOMER_929,2023-03-01,793.0,Netbanking,"623, Chhotaudepur, Vadodara, GUJARAT",111369
44985,ORDER_9998,CUSTOMER_1565,2022-04-06,2961.0,Wallet,"1703, Mohammdabad Gohana, Mau, UTTAR PRADESH",112309


In [29]:
opt = analyse_with_ai(D)
opt

SyntaxError: invalid syntax (<unknown>, line 1)

In [4]:
from collections import Counter
print(Counter("hello world").most_common(3))

[('l', 3), ('o', 2), ('h', 1)]


In [ ]:


class PostgresDB:
    def __init__(self, db_url):
        self.db_url = os.getenv('DB_HOST')
        self.conn = None
        self.cursor = None
    
    def connect_with_url(self):
        """Establish connection to the PostgreSQL database."""
        self.conn = psycopg2.connect(self.db_url)
        self.cursor = self.conn.cursor()
    
    def __enter__(self):
        """Context manager entry."""
        self.connect_with_url()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
    
    def upsert(self, table_name, data_dict):
        """Insert or update a row in the specified table."""
        # Check if 'id' is in the dict
        if 'id' in data_dict:
            columns = data_dict.keys()
            values = tuple(data_dict.values())
            set_clause = ', '.join([f"{key} = %s" for key in columns])
            query = sql.SQL(f"""
                INSERT INTO {sql.Identifier(table_name)} ({', '.join(columns)}) 
                VALUES ({', '.join(['%s'] * len(columns))})
                ON CONFLICT (id) 
                DO UPDATE SET {set_clause};
            """)
            self.cursor.execute(query, values)
        else:
            raise ValueError("Data dictionary must contain an 'id' field.")
        self.conn.commit()

    def delete(self, table_name, id):
        """Delete a row in the specified table by id."""
        query = sql.SQL("DELETE FROM {table} WHERE id = %s;").format(table=sql.Identifier(table_name))
        self.cursor.execute(query, (id,))
        self.conn.commit()
    
    def get(self, table_name, id):
        """Get a row in the specified table by id."""
        query = sql.SQL("SELECT * FROM {table} WHERE id = %s;").format(table=sql.Identifier(table_name))
        self.cursor.execute(query, (id,))
        result = self.cursor.fetchone()
        return result
    
    def get_all(self, table_name):
        """Get all rows in the specified table."""
        query = sql.SQL("SELECT * FROM {table};").format(table=sql.Identifier(table_name))
        self.cursor.execute(query)
        result = self.cursor.fetchall()
        return result
    
    def run_sql(self, sql_statement):
        """Run a custom SQL statement."""
        self.cursor.execute(sql_statement)
        self.conn.commit()
    
    def get_table_definitions(self, table_name):
        """Get table definitions in 'CREATE TABLE' format."""
        query = sql.SQL("""
            SELECT column_name, data_type, character_maximum_length
            FROM information_schema.columns
            WHERE table_name = %s
            ORDER BY ordinal_position;
        """)
        self.cursor.execute(query, (table_name,))
        columns = self.cursor.fetchall()
        
        create_table_statement = f"CREATE TABLE {table_name} (\n"
        for column in columns:
            col_name = column[0]
            data_type = column[1]
            max_length = column[2]
            if max_length:
                create_table_statement += f"    {col_name} {data_type}({max_length}),\n"
            else:
                create_table_statement += f"    {col_name} {data_type},\n"
        create_table_statement = create_table_statement.rstrip(",\n") + "\n);"
        return create_table_statement
    
    def get_all_table_names(self):
        """Get all table names in the database."""
        query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
        self.cursor.execute(query)
        result = self.cursor.fetchall()
        return [table[0] for table in result]
    
    def get_table_definitions_for_prompt(self):
        """Get 'CREATE TABLE' definitions for all tables for LLM prompt."""
        table_names = self.get_all_table_names()
        table_definitions = ""
        for table in table_names:
            table_definitions += f"Table: {table}\n"
            table_definitions += self.get_table_definitions(table) + "\n\n"
        return table_definitions


In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("prompt",help = "The prompt for AI")
    args = parser.parse_arg()
    with Postgresmanager() as db:
        db.connect_with_url

if __name__ == '__main__':
    main()